In [27]:
import pandas as pd
from pathlib import Path
from datetime import datetime
import numpy as np
import json
import matplotlib.pyplot as plt

In [1]:
test = [{"a": 1, "b": 2}, {"a": 3, "b": 4}, {"a": 5, "b": 6}]

In [5]:
x = test.__repr__()
print(type(x))
print(x)

<class 'str'>
[{'a': 1, 'b': 2}, {'a': 3, 'b': 4}, {'a': 5, 'b': 6}]


In [28]:
run_folder = Path("../tmp/swarm").resolve()
runs = {0: run_folder}
print(runs)

{0: WindowsPath('D:/00Repos/GPT-Swarm/tmp/swarm')}


In [29]:
def load_run(runs, run_id=None):
    """Logs format:
    {"time": "2023-04-15 12:20:27,477", "level": "INFO", "agent_id": -1, "cycle": -1, "step": "swarm", "message": "Created 2 agents with roles: ['python developer' 'python developer']"}
    {"time": "2023-04-15 12:20:27,477", "level": "INFO", "agent_id": -1, "cycle": -1, "step": "swarm", "message": "Agents roles:\n[['p' 'p']]"} 
    """
    if run_id is None:
        run = runs[-1]
    else:
        run = runs[run_id]
    print(f"Loading run {run_id}: {run}")

    log_file = run / "swarm.json"

    # parse in pandas
    df = pd.read_json(log_file, lines=True)
    df["time"] = pd.to_datetime(df["time"])
    df["agent_id"] = df["agent_id"].astype(int)
    df["cycle"] = df["cycle"].astype(int)
    df["step"] = df["step"].astype(str)
    df["message"] = df["message"].astype(str)
    df["level"] = df["level"].astype(str)

    # unique agents
    agents = df["agent_id"].unique()
    print(f"Found {len(agents)} agents (-1 is swarm itself): {agents}")

    # unique cycles
    cycles = df["cycle"].unique()
    print(f"Found {len(cycles)} cycles: {cycles}")

    # unique steps
    steps = df["step"].unique()
    print(f"Found {len(steps)} steps: {steps}")
    return df

def pretty_display(df, level=None, agent_id=None, cycle=None, step=None, message=None):
    """Pretty display of the logs"""
    if level is not None:
        df = df[df["level"] == level]
    if agent_id is not None:
        df = df[df["agent_id"] == agent_id]
    if cycle is not None:
        df = df[df["cycle"] == cycle]
    if step is not None:
        df = df[df["step"] == step]
    if message is not None:
        df = df[df["message"] == message]

    
    for _, row in df.iterrows():
        print("====================================================================================================================================================")
        print("====================================================================================================================================================")
        print("====================================================================================================================================================")
        print("====================================================================================================================================================")
        print(f"{row['time']} {row['level']} {row['agent_id']} {row['cycle']} {row['step']}")
        print(row['message'])
    return df
              

In [30]:
df = load_run(runs, run_id=0)

Loading run 0: D:\00Repos\GPT-Swarm\tmp\swarm
Found 7 agents (-1 is swarm itself): [-1  0  1  2  3  4  5]
Found 18 cycles: [-1  0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]
Found 2 steps: ['swarm' 'init']


C:\Users\nicel\AppData\Local\Temp\ipykernel_22060\2463234948.py:16: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["time"] = pd.to_datetime(df["time"])


In [31]:
df_i = pretty_display(df, agent_id=0, cycle=2)
df_i

2023-04-29 17:06:05.311000 INFO 0 2 init
Got task: Act as:
professional venture capital agency, who has a proven track reckord of consistently funding successful startups
Gloabl goal:
A new startup just send us their pitch. Find if the startup is worth investing in. The startup is in the space of brain computer interfaces.
Their value proposition is to provide objective user experience research for new games beased directly on the brain activity of the user.
They have a proprietary algorithm to predict emotions and user experience based on brain activity.
They don't develop the games themselves, but they provide the data to game developers.
They don't develop any hardware, but they provide the software to collect and analyze the data.
More information about them: 'https://brainamics.de', 'https://www.linkedin.com/company/thebrainamics/'

Your specific task is:
Briefly describe the technology for the non-tech audience. Include links to the main articles in the field. of type: breakdown_

,time,level,agent_id,cycle,step,message
30,2023-04-29 17:06:05.311,INFO,0,2,init,Got task: Act as:\nprofessional venture capita...
35,2023-04-29 17:06:11.375,DEBUG,0,2,init,To shared memory: Task 'Act as:\nprofessional ...
137,2023-04-29 17:40:25.097,DEBUG,0,2,init,Got task: ca55c7d7-012e-433e-b9bb-b5715c0fcd62
139,2023-04-29 17:40:25.159,DEBUG,0,2,init,Got task: c28d713f-0974-48a9-8d33-da62ee3e9ce0
147,2023-04-29 17:40:28.177,DEBUG,0,2,init,Got task: 9c3a9f33-1dc7-494f-8b7e-b17a80ddf83b
150,2023-04-29 17:40:37.492,INFO,0,2,init,Task:\n'Act as:\nprofessional venture capital ...
154,2023-04-29 17:40:39.663,DEBUG,0,2,init,"To shared memory: As an AI language model, I d..."
159,2023-04-29 17:40:48.802,DEBUG,0,2,init,To shared memory: Brainamics has a unique valu...
315,2023-04-29 17:48:14.352,DEBUG,0,2,init,Got task: fd2a1762-b444-4ed5-a990-133ab62f5eaa
319,2023-04-29 17:48:21.926,INFO,0,2,init,Task:\n'Act as:\nprofessional venture capital ...
